# Observaciones convencionales horarias

Se extraen los datos climáticos horarios de las estaciones automáticas de la AEMET generados en las 24 h últimas. Se pueden descargar todas las estaciones o de las estaciones determinadas.

Si se hiciera recurrentemente se 

In [16]:
# Archivo donde está guardada la API key
ruta_API = 'F:/Series/AEMET/apikey_AEMET.txt'
# Carpeta en la que se guardarán las series
ruta_series = 'F:/Series/AEMET/Clima/Horario/'
# Estaciones a importar: todas o una lista de las que se quiera
codigos = ['todas']
# Archivo con los datos de las estaciones automáticas de la AEMET
ruta_estaciones = 'F:/Series/AEMET/Clima/Horario/Estaciones_Automaticas.csv'

In [17]:
import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import pandas as pd
import numpy as np
import datetime
import time

from pathlib import Path

## 1. Estaciones automáticas

Se importa el listado de las estaciones automáticas de la AEMET en toda España. Estas estaciones son las susceptibles de aportar datos por este medio.

In [18]:
estaciones =pd.read_csv(ruta_estaciones)
estaciones.drop(['FID'], axis=1, inplace=True)
estaciones.columns = ['idema', 'ubi', 'PROVINCIA', 'alt', 'COORD_X', 'COORD_Y', 'VAR_OBSERV', 'DATUM', 'TIPO']
estaciones.sort_values('idema', inplace=True)
estaciones.set_index('idema', drop=False, inplace=True)

# Corregir caracteres conflictivos en el nombre de las estaciones
for i in estaciones.index:
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('/', '-')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('"', '')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Á', 'A')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('É', 'E')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Í', 'I')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ó', 'O')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ú', 'U')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('À', 'A')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('È', 'E')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ç', 'C')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ñ', 'N')

print('Nº de estaciones automáticas de la AEMET: ', estaciones.shape[0])

Nº de estaciones automáticas de la AEMET:  852


## 1. Extraer todas las estaciones

Se extraen los datos climáticos horarios disponibles de las últimas 24 h.

### 1.1 Funciones

In [19]:
def extract_hourly_all(API_path, series_path):
    """It extracts hourly climatic data from AEMET's API for all the stations available during the last 24 h.
    
    Parameters:
    -----------
    API_path:     string. Path where the API key text file is saved, including the name and file extension.
    series_path:  string. File path where the previous data is saved (or where the new series will be created). 
    """
    
    # Extraer de AEMET los datos del último día
    # -----------------------------------------
     # Carga la api key 
    api_key = open(API_path).read().rstrip()
    querystring = {"api_key": api_key}
  
    # Obtenemos información de todas las estaciones disponibles
    url = "https://opendata.aemet.es/opendata/api/observacion/convencional/todas"
    
    iterate = True
    
    while iterate:
        # Peticion de datos a la url indicada
        r = requests.get(url, params=querystring, verify=False)
        # Si no me deja hacer la conexión, la repito  
        iterate = (r.status_code == requests.codes.too_many_requests)
        print(r.json())
        
        # Chequeo si la petición ha ido bien    
        if r.status_code == requests.codes.ok:
            # Hago la petición para obtener los datos
            data_url = r.json()['datos']
            r_data = requests.get(data_url, params=querystring, verify=False)

            # INCONSISTENCIA DE LA API:
            # Cuando no encuentra datos en el rango seleccionado, la API devuelve
            # que el status code es 200 (todo ok) y devuelve un json con el error
            # cuando encuentra, no hay atributo estado            
            try:
                estado = r_data.json()['estado']
            except:
                estado = 200

            # Si ha ido bien guardo los datos
            if estado == requests.codes.ok:
                #print(r_data.json())
                data = (r_data.json())
                # Crear una lista con cada una de las entradas de los datos brutos convertidos en data frames
                ls = []
                for i in np.arange(len(data)):
                    ls.append(pd.DataFrame(data[i], index=[i]))
                # Combinar los data frames de la lista en un unico data frame y definir la hora como índice
                data = pd.concat(ls)
                data.fint = pd.to_datetime(data.fint, yearfirst=True)
                data.set_index('fint', inplace=True)
                # Reorganizar algunas columnas
                col = df.columns.tolist()
                col.remove('alt')
                col.remove('idema')
                col.remove('lat')
                col.remove('lon')
                col.remove('ubi')
                col = ['idema', 'ubi', 'lat', 'lon', 'alt'] + col
                data = data[col]
            else:
                print(r_data.json()['descripcion'])
        else:
            print(r.json()['descripcion'])

        time.sleep(60/45)
    
    # Nombre de las estaciones
    estaciones = data.idema.unique()
    
    for i in range(len(estaciones)):
        # Extraer las filas correspondientes a una estación
        data_stn = data.loc[data.idema == estaciones[i], :]
        cod = data_stn.idema[0]
        name = data_stn.ubi[0]
        # Corregir caracteres conflictivos en el nombre de las estaciones
        name = name.replace('/', '-')
        name = name.replace('"', '')
        name = name.replace('Á', 'A')
        name = name.replace('É', 'E')
        name = name.replace('Í', 'I')
        name = name.replace('Ó', 'O')
        name = name.replace('Ú', 'U')
        name = name.replace('À', 'A')
        name = name.replace('È', 'E')
        name = name.replace('Ò', 'O')
        name = name.replace('Ü', 'U')
        name = name.replace('Ç', 'C')
        name = name.replace('Ñ', 'N')
        name = name.replace('?', ' ')

        file = Path(series_path + cod + '_' + name + '.csv')
        if file.is_file():
            # Importar la serie preexistente
            serie = pd.read_csv(file, encoding='latin1', na_values='NaN')
            serie.fint = pd.to_datetime(serie.fint)
            serie.set_index('fint', inplace=True, drop=True)
            # Unir las nuevas observaciones
            serie = serie.append(data_stn[df_stn.index > serie.index[-1]], verify_integrity=True)
            # Exportar la serie
            serie.to_csv(series_path + cod + '_' + name + '.csv', index=True, na_rep='NaN')
            #del serie, df_stn#, raw_data
        else:
            data_stn.to_csv(series-path + cod + '_' + name + '.csv', index=True, na_rep='NaN')
            #del data_stn#, raw_data

### 1.2 Extraer datos

In [20]:
while True:
    extract_hourly_all(ruta_API, ruta_series)
    time.sleep(23 * 3600)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/9c512188', 'metadatos': 'https://opendata.aemet.es/opendata/sh/55c2971b'}


NameError: name 'df' is not defined

## 2. Extraer una única estación

### 2.1 Funciones

In [ ]:
def extract_data_stn(cod, name, API_path):
    """It extracts hourly climatic data from AEMET's API.
    
    Parameters:
    -----------
    cod:      string. Station code. It must match a index value in the data frame 'stations'.
    stations: data frame. Matrix of available stationes. Its index must be the station's code and it must have a field called
              'ubi' which represents the station's name
    API_path: string. The path where the API key text file is saved, including the name and file extension.
    """
    
    # Extraer de AEMET los datos del último día
    # -----------------------------------------
    # Carga la api key 
    api_key = open(API_path).read().rstrip()
    querystring = {"api_key": api_key}

    raw_data = []
    
    # Mostrar en pantalla la estación de trabajo
    print()
    print('Estación: ', cod, ' - ', name)
    
    # Obtenemos información de todas las estaciones disponibles
    url = ("https://opendata.aemet.es/opendata/api/observacion/convencional/datos/estacion//{station}".format(
            station=cod)
          )
    
    iterate = True
    
    while iterate:
        # Peticion de datos a la url indicada
        r = requests.get(url, params=querystring, verify=False)
        # Si no me deja hacer la conexión, la repito  
        iterate = (r.status_code == requests.codes.too_many_requests)
        print(r.json())
        
        # Chequeo si la petición ha ido bien    
        if r.status_code == requests.codes.ok:
            # Hago la petición para obtener los datos
            data_url = r.json()['datos']
            r_data = requests.get(data_url, params=querystring, verify=False)

            # INCONSISTENCIA DE LA API:
            # Cuando no encuentra datos en el rango seleccionado, la API devuelve
            # que el status code es 200 (todo ok) y devuelve un json con el error
            # cuando encuentra, no hay atributo estado            
            try:
                estado = r_data.json()['estado']
            except:
                estado = 200

            # Si ha ido bien guardo los datos
            if estado == requests.codes.ok:
                #print(r_data.json())
                raw_data.extend(r_data.json())
            else:
                print(r_data.json()['descripcion'])
        else:
            print(r.json()['descripcion'])

        time.sleep(60/45)
    return raw_data

In [ ]:
def generate_serie_stn(cod, name, raw_data, path):
    """It joins the past and present series of hourly climatic data. If there's not previous data, it simply creates a new
    csv file with the hourly data from the last 24 h.
    
    Parameters:
    -----------
    cod:      string. Station code.
    name:     string. Station name.
    raw_data: data frame. The result of function 'extract_hourly_data'.
    path:     string. File path where the previous data is saved (or where the new series will be created).
    """
    # Reorganizar la serie si hay observaciones o hay datos previos
    # -------------------------------------------------------------     
    # Asegurar que el nuevo data frame tiene 24 filas (con NaN si no hay obervación)
    start = datetime.datetime.now().replace(microsecond=0,second=0,minute=0) - datetime.timedelta(hours=24)
    horas = pd.DatetimeIndex(freq='H', start=start, periods=24)
    columns = ['idema', 'ubi', 'lat', 'lon', 'alt', 'prec', 'ta', 'tamax', 'tamin', 'vv', 'dv',
                              'vmax', 'dmax', 'hr']
    df2 = pd.DataFrame(index=horas, columns=columns)
    df2.index.name = 'hora'
    # Si hay datos disponibles de las últimas horas introducir dichos datos
    if len(raw_data) > 0:
        # Crear una lista de data frames con los datos de cada hora
        dfs = []
        for i in np.arange(len(raw_data)):
            dfs.append(pd.DataFrame(raw_data[i], index=[i]))

        # Combinar los data frames de la lista en un unico data frame
        df = pd.concat(dfs)
        df.fint = pd.to_datetime(df.fint, yearfirst=True)
        df.set_index('fint', inplace=True)
        df2.loc[df.index, :] = df
        #del df, dfs
    
    file = Path(path + cod + '_' + name + '.csv')
    if file.is_file():
        # Importar la serie preexistente
        serie = pd.read_csv(file, encoding='latin1', na_values='NaN')
        serie.hora = pd.to_datetime(serie.hora)
        serie.set_index('hora', inplace=True, drop=True)
        # Unir las nuevas observaciones
        serie = serie.append(df2[df2.index > serie.index[-1]], verify_integrity=True)
        # Exportar la serie
        serie.to_csv(path + cod + '_' + name + '.csv',
               index=True, na_rep='NaN')
        del serie, df2, raw_data
    else:
        if len(raw_data) > 0:
            df2.to_csv(path + cod + '_' + name + '.csv',
               index=True, na_rep='NaN')
            del df2, raw_data

### 2.2 Aplicarlo a todas las estaciones en bucle

In [ ]:
if codigos[0] != 'todas':
    for stn in codigos:
        # Nombre de la estación
        nombre = estaciones.loc[stn, ubi]
        # Extraer datos
        data_stn = extract_data_stn(stn, nombre, ruta_API)
        # Generar la serie
        generate_serie_stn(stn, nombre, data_stn, ruta_series)

In [ ]:
import schedule